### Imports

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
import pandas as pd

# 1. Download the Dataset and Understand the Format

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
google_drive = GoogleDrive(gauth)
downloaded = google_drive.CreateFile({'id':"1v4zSFOdfbFyMFalPVB95M0atJDryKRYb"})
downloaded.GetContentFile('Crema.zip')        
!unzip -qq /content/Crema.zip -d /content/dataset

In [ ]:
path = "/content/dataset/Crema/"
crema_directory = os.listdir(path)

In [ ]:
emotions = []
file_path = []

In [ ]:
for file in crema_directory:
  file_path.append(path + file)
  emotion = file.split('_')
  if emotion[2] == 'SAD':
      emotions.append('sad')
  elif emotion[2] == 'ANG':
      emotions.append('angry')
  elif emotion[2] == 'DIS':
      emotions.append('disgust')
  elif emotion[2] == 'FEA':
      emotions.append('fear')
  elif emotion[2] == 'HAP':
      emotions.append('happy')
  elif emotion[2] == 'NEU':
      emotions.append('neutral')
  else:
      emotions.append('unknown emotion')

In [ ]:
Emotions_df = pd.DataFrame(emotions, columns=['Emotion'])
path_df = pd.DataFrame(file_path, columns=['Path'])
Crema_df = pd.concat([path_df, Emotions_df], axis=1)
Crema_df.head()

# 2. Create the Feature Space

# 3. Building the Model

# 4. Big Picture